In [1]:
import pandas as pd
import sys
import statsmodels.api as sm
import datetime
import numpy as np

In [2]:
df = pd.read_csv('StockData.csv',index_col=0)
df.rename(columns={'Adj Close': 'Index'}, inplace=True)
index_oil_gold_gas = df.iloc[:,-4:]

In [3]:
# beta in regression is what a 1-unit increase in the specific variable does to the dependent variable
# significance: .x means x% this was a random event, 0% means no random (causation)
# when we get betas they assume we were holding everything else constant

# X = sm.add_constant(index_oil_gold_gas)
# model = sm.OLS(y,X).fit()
# model.summary()

In [4]:
# create a function that take stock name as para, return a dataframe with significant betas
def regression(stock):
    y = df[stock]
    X = sm.add_constant(index_oil_gold_gas)
    model = sm.OLS(y,X).fit()
    # only display betas with p-values under 5%
    sig_betas = model.pvalues < .05
    stock_betas = model.params[sig_betas].to_frame()
    stock_betas.rename(columns={0:stock},inplace=True)
    return stock_betas

In [15]:
mega = pd.DataFrame()
for stock in df.columns[:-4]:
    frame = regression(stock)
    mega = pd.concat([mega, frame],axis=1)
mega = mega.reindex(['const','Index','Oil','Gold','NaturalGas'])
mega.to_csv('RegressionMatrix1.csv')